In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow.keras as keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras import optimizers
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing import image
import scipy

In [2]:
#set epochs, batch size
num_epochs = 1
batches = 16

#set directories
workDir = "/Users/czkaiweb/Research/DisappTrksML/VAE/"
saveDir = workDir
dataDir = "/Users/czkaiweb/Research/DisappTrkMLData/converted/"

#load data
data_e = np.load(dataDir+'e_DYJets50V3_norm_40x40_nonscaled_tiny.npy')
data_bkg = np.load(dataDir+'bkg_DYJets50V3_norm_40x40_nonscaled_tiny.npy')
test_e = np.load(dataDir+'e_DYJets50V3_norm_40x40_nonscaled_tinyTest.npy')
test_bkg = np.load(dataDir+'bkg_DYJets50V3_norm_40x40_nonscaled_tinyTest.npy')

#save 20% of data for testing
test_index_e = int(len(data_e)/5)
test_index_bkg = int(len(data_bkg)/5)

#shuffle e and bkg data
indicies = np.arange(len(data_e))
np.random.shuffle(indicies)
data_e = data_e[indicies]
indicies = np.arange(len(data_bkg))
np.random.shuffle(indicies)
data_bkg = data_bkg[indicies]

#test data
test_e = data_e[:test_index_e]
test_bkg = data_e[:test_index_bkg]

data = np.concatenate([data_e[test_index_e:], data_bkg[test_index_bkg:]])
classes = np.concatenate([np.ones(len(data_e[test_index_e:])), np.zeros(len(data_bkg[test_index_bkg:]))])
print(len(data_e), len(data_bkg))

#shuffle data
indicies = np.arange(data.shape[0])
np.random.shuffle(indicies)
data = data[indicies]
classes = classes[indicies]
print(data.shape[0], "Number of samples")
print(np.shape(data))

x_train, x_val, y_train, y_val = train_test_split(data, classes, test_size=0.15, random_state=0)


(119, 495)
(492, 'Number of samples')
(492, 5, 40, 40)


In [23]:
class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon


def Encode(dim_list,latent_dim):
    x_dim,y_dim,nlayers = dim_list
    encoder_inputs = keras.Input(shape=(x_dim, x_dim, nlayers))
    x = layers.Conv2D(32, 3, activation="relu", strides=2, padding="same")(encoder_inputs)
    x = layers.Conv2D(64, 3, activation="relu", strides=2, padding="same")(x)
    x = layers.Flatten()(x)
    x = layers.Dense(16, activation="relu")(x)
    z_mean = layers.Dense(latent_dim, name="z_mean")(x)
    z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
    z = Sampling()([z_mean, z_log_var])
    encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")
    encoder.summary()
    return encoder

def Decode(dim_list,latent_dim):
    x_dim,y_dim,nlayers = dim_list
    latent_inputs = keras.Input(shape=(latent_dim,))
    x = layers.Dense( x_dim/4* y_dim/4 * 64, activation="relu")(latent_inputs)
    x = layers.Reshape((x_dim/4, y_dim/4, 64))(x)
    x = layers.Conv2DTranspose(64, 3, activation="relu", strides=2, padding="same")(x)
    x = layers.Conv2DTranspose(32, 3, activation="relu", strides=2, padding="same")(x)
    decoder_outputs = layers.Conv2DTranspose(nlayers, 3, activation="sigmoid", padding="same")(x)
    decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")
    decoder.summary()
    return decoder

class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder

    def train_step(self, data):
        if isinstance(data, tuple):
            data = data[0]
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = encoder(data)
            reconstruction = decoder(z)
            reconstruction_loss = tf.reduce_mean(
                keras.losses.binary_crossentropy(data, reconstruction)
            )
            reconstruction_loss *= 40 * 40
            kl_loss = 1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var)
            kl_loss = tf.reduce_mean(kl_loss)
            kl_loss *= -0.5
            loss = reconstruction_loss + kl_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.loss = loss
        # Update metrics (includes the metric that tracks the loss)
        self.compiled_metrics.update_state(y, y_pred)
        # Return a dict mapping metric names to current value
        return {
            "loss": loss,
            "reconstruction_loss": reconstruction_loss,
            "kl_loss": kl_loss,
        }
        return {m.name: m.result() for m in self.metrics}

    

In [25]:
encoder = Encode([40,40,5],5)
decoder = Decode([40,40,5],5)

vae = VAE(encoder, decoder)
vae.compile(optimizer=keras.optimizers.Adam())
vae.fit(data, epochs=30, batch_size=128)

Model: "encoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_29 (InputLayer)           [(None, 40, 40, 5)]  0                                            
__________________________________________________________________________________________________
conv2d_28 (Conv2D)              (None, 20, 20, 32)   1472        input_29[0][0]                   
__________________________________________________________________________________________________
conv2d_29 (Conv2D)              (None, 10, 10, 64)   18496       conv2d_28[0][0]                  
__________________________________________________________________________________________________
flatten_14 (Flatten)            (None, 6400)         0           conv2d_29[0][0]                  
____________________________________________________________________________________________

ValueError: The model cannot be compiled because it has no loss to optimize.

In [ ]:
def plot_latent(encoder, decoder):
    # display a n*n 2D manifold of digits
    n = 30
    digit_size = 28
    scale = 2.0
    figsize = 15
    figure = np.zeros((digit_size * n, digit_size * n))
    # linearly spaced coordinates corresponding to the 2D plot
    # of digit classes in the latent space
    grid_x = np.linspace(-scale, scale, n)
    grid_y = np.linspace(-scale, scale, n)[::-1]

    for i, yi in enumerate(grid_y):
        for j, xi in enumerate(grid_x):
            z_sample = np.array([[xi, yi]])
            x_decoded = decoder.predict(z_sample)
            digit = x_decoded[0].reshape(digit_size, digit_size)
            figure[
                i * digit_size : (i + 1) * digit_size,
                j * digit_size : (j + 1) * digit_size,
            ] = digit

    plt.figure(figsize=(figsize, figsize))
    start_range = digit_size // 2
    end_range = n * digit_size + start_range + 1
    pixel_range = np.arange(start_range, end_range, digit_size)
    sample_range_x = np.round(grid_x, 1)
    sample_range_y = np.round(grid_y, 1)
    plt.xticks(pixel_range, sample_range_x)
    plt.yticks(pixel_range, sample_range_y)
    plt.xlabel("z[0]")
    plt.ylabel("z[1]")
    plt.imshow(figure, cmap="Greys_r")
    plt.show()


plot_latent(encoder, decoder)

In [ ]:
def RecoErr(img):
    input_img = img.reshape(1, 40, 40, 5)
    pred = autoencoder.predict(input_img)
    img_flat = img.reshape(1600, 5)
    pred_flat = pred.reshape(1600, 5)
    err = np.zeros(5)
    for i in range(5):
        err[i] = np.linalg.norm(img_flat[:, i] - pred_flat[:, i], axis=-1)
    return err

In [ ]:
e_ecal_err = []
e_ecalS_err = []
e_hcal_err = []
e_muons_err = []
e_err = [e_ecal_err, e_ecalS_err, e_hcal_err, e_muons_err]
bkg_ecal_err = []
bkg_ecalS_err = []
bkg_hcal_err = []
bkg_muons_err = []
bkg_err = [bkg_ecal_err, bkg_ecalS_err, bkg_hcal_err, bkg_muons_err]
e_indicies = np.arange(int(test_e.shape[0]/2))
bkg_indicies = np.arange(int(test_bkg.shape[0]/2))
trials_e = np.random.choice(e_indicies, size=(500))
trials_bkg = np.random.choice(bkg_indicies, size=(500))
for i in range(500):
    this_e_err = RecoErr(test_e[trials_e[i]])
    this_bkg_err = RecoErr(test_bkg[trials_bkg[i]])
    for j in range(4):
        e_err[j].append(this_e_err[j+1])
        bkg_err[j].append(this_bkg_err[j+1])

print("done again")

In [ ]:
num_bins = 100
fig, my_plt = plt.subplots(2, 4, figsize=(15,10))
my_plt[0,0].hist(e_err[0], bins=num_bins)
my_plt[0,0].set_title("ECAL Electron")
my_plt[0,1].hist(e_err[1], bins = num_bins)
my_plt[0,1].set_title("ECAL Preshower Electron")
my_plt[0,2].hist(e_err[2], bins = num_bins)
my_plt[0,2].set_title("HCAL Electron")
my_plt[0,3].hist(e_err[3], bins = num_bins)
my_plt[0,3].set_title("Muons Electron")
my_plt[1,0].hist(bkg_err[0], bins=num_bins)
my_plt[1,0].set_title("ECAL Background")
my_plt[1,1].hist(bkg_err[1], bins = num_bins)
my_plt[1,1].set_title("ECAL Preshower Bakckground")
my_plt[1,2].hist(bkg_err[2], bins = num_bins)
my_plt[1,2].set_title("HCAL Background")
my_plt[1,3].hist(bkg_err[3], bins = num_bins)
my_plt[1,3].set_title("Muons Background")

In [ ]:
std_e = np.zeros(4)
std_bkg = np.zeros(4)
mean_e = np.zeros(4)
mean_bkg = np.zeros(4)

fig2, ax = plt.subplots(2,4, figsize = (15,10))

for i in range(4):
    fit_e = scipy.stats.moyal.fit(e_err[i])
    fit_bkg = scipy.stats.moyal.fit(bkg_err[i])
    moy = scipy.stats.moyal(fit_e[0], fit_e[1])
    moy_bkg = scipy.stats.moyal(fit_bkg[0], fit_bkg[1])
    x = np.linspace(moy.ppf(0.001), moy.ppf(0.999), 200)
    x_bkg = np.linspace(moy_bkg.ppf(0.001), moy.ppf(0.999), 200)
    ax[0,i].plot(x, moy.pdf(x), color="red")
    ax[1,i].plot(x_bkg, moy_bkg.pdf(x_bkg), color="red")
    std_e[i] = moy.std()
    std_bkg[i] = moy_bkg.std()
    mean_e[i] = moy.mean()
    mean_bkg[i] = moy_bkg.mean()

print(fit_e) 
print(std_e)
print(mean_e)
#m_ecal = scipy.stats.moyal(fit_e[0])
#my_plt[0,0].plot(m_ecal, color="red")
print("fine")

In [ ]:
def testImg(img):
    err = RecoErr(img)
    this_std_e = abs(err[1:] - mean_e) / std_e
    this_std_bkg = abs(err[1:] - mean_bkg) / std_bkg
    sim_count = 0
    is_e = False
    small_std = 10e6
    for i in range(4):
        #if(this_std_e[i] < this_std_bkg[i]): sim_count +=1
        #else: sim_count += -1
        if(this_std_e[i] < small_std): 
            small_std = this_std_e[i]
            is_e = True
        if(this_std_bkg[i] < small_std):
            small_std = this_std_bkg[i]
            is_e = False
    #if(sim_count >= 0): is_e = True
    #if(sim_count == 0): return this_std_e, this_std_bkg
    #if(sim_count < 0): is_e = False
    return this_std_e, this_std_bkg, is_e

In [ ]:
num_e_test = int(len(test_e)/2)
num_bkg_test = int(len(test_bkg)/2)
e_evts = 0
bkg_evts = 0
for i in range(num_e_test):
    e_std, bkg_std, is_e = testImg(test_e[num_e_test+i])
    #print(e_std, bkg_std, is_e)
    if(is_e == True): e_evts += 1
    else: bkg_evts += 1
print(e_evts, bkg_evts)
e_evts = 0
bkg_evts = 0
for i in range(num_bkg_test):
    e_std, bkg_std, is_e = testImg(test_bkg[num_bkg_test+i])
    if(is_e == True): e_evts += 1
    else: bkg_evts += 1
print(e_evts, bkg_evts)
